# Crawl Evidence Central from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup
import requests

from models.crawl_utils import get_page, save_page

In [ ]:
# config
host = 'https://evidencecentral.org/sitemap.xml'
base_dir = '../data/raw/evidence_central'
bs_parser = 'xml'
delay_seconds = 5

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    

In [ ]:
def extract_text(xml_content):
    #Parse the XML content using BeautifulSoup
    soup = BeautifulSoup(xml_content, 'html.parser')

    #Find all <loc> tags in the XML
    loc_tags = soup.find_all('loc')

    #Extract and store the text inside each <loc> tag
    loc_texts = [loc.get_text() for loc in loc_tags]

    return loc_texts

In [ ]:
def get_path(url):
    path_components = urlparse(url).path.split('/')
    # print(path_components)
    return os.path.join(base_dir, f"{path_components[-1]}.json")

In [ ]:

def get_http(requests, host):
    headers = {
        'Content-Type': 'application/json',
    }
    response = requests.get(host, headers=headers)

    return response

    # Note: json_data will not be serialized by requests
    # exactly as it was in the original request.
    #data = '{"tblnm":"virctnrnyvw","templateId":116,"parameters":"{\\"tblEty\\":314,\\"prnFomtplhdrID\\":116,\\"path\\":\\"/recency/evidence/make-a-record\\",\\"templateKey\\":\\"PAGES/CONTENT_RECENCY_VIEWER\\",\\"uiKey\\":\\"AF24A18C-F8BD-4742-AAF8-955E7AF86206\\",\\"dynamicStateGuid\\":\\"d0450ef1-aa1e-4240-8ef5-a1721f3e3564\\",\\"isFormValid\\":true,\\"fileTemplate\\":\\"recency\\"}","formValues":"{\\"tblEty\\":314}"}'
    #response = requests.post('https://evidencecentral.org/api/fileTemplate/process/html', headers=headers, data=data)

In [ ]:
def call_http(requests, path):
    path_route = path.replace("https://evidencecentral.org", "")
    print(path_route)
    headers = {
        'Content-Type': 'application/json',
    }

    json_data = {
        'tblnm': 'virctnrnyvw',
        'templateId': 116,
        'parameters': '{"tblEty":314,"prnFomtplhdrID":116,"path":"'+path_route+'","templateKey":"PAGES/CONTENT_RECENCY_VIEWER","uiKey":"AF24A18C-F8BD-4742-AAF8-955E7AF86206","dynamicStateGuid":"d0450ef1-aa1e-4240-8ef5-a1721f3e3564","isFormValid":true,"fileTemplate":"recency"}',
        'formValues': '{"tblEty":314}',
    }
    response = requests.post('https://evidencecentral.org/api/fileTemplate/process/html', headers=headers, json=json_data)

    return response

    # Note: json_data will not be serialized by requests
    # exactly as it was in the original request.
    #data = '{"tblnm":"virctnrnyvw","templateId":116,"parameters":"{\\"tblEty\\":314,\\"prnFomtplhdrID\\":116,\\"path\\":\\"/recency/evidence/make-a-record\\",\\"templateKey\\":\\"PAGES/CONTENT_RECENCY_VIEWER\\",\\"uiKey\\":\\"AF24A18C-F8BD-4742-AAF8-955E7AF86206\\",\\"dynamicStateGuid\\":\\"d0450ef1-aa1e-4240-8ef5-a1721f3e3564\\",\\"isFormValid\\":true,\\"fileTemplate\\":\\"recency\\"}","formValues":"{\\"tblEty\\":314}"}'
    #response = requests.post('https://evidencecentral.org/api/fileTemplate/process/html', headers=headers, data=data)

In [ ]:
# status_code, html = get_page(host, delay_seconds)
response = get_http(requests, host)
xml_content = response.content
xml_links = extract_text(xml_content)
print(xml_links[2])


In [ ]:
for url in xml_links:
    # print(url)
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    html = call_http(requests, url)
    print(html.content)
    # if status_code != 200:
    #     print("Error!", status_code , url)
    #     continue
    save_page(path_file,url,html.content.decode('utf-8'))
    
print("End")